In [1]:
%load_ext autoreload
%autoreload 2
from new_aeb_gplvm import *
from utils.data_generator import DataGenerator
from utils.myutils import Utils
from tqdm import trange
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import random
from tqdm import tqdm_notebook

random.seed(42)
import json
import os
import pandas as pd

datagenerator = DataGenerator()
utils = Utils()

2023-11-29 13:02:53.753167: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 13:02:53.753201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 13:02:53.754290: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 13:02:53.760837: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 13:02:54.593608: W tensorflow/comp

In [2]:
EXPERIMENTS_FILE = "experiments/noisy/004_irrelevant_anomaly_all_results.json"
with open(EXPERIMENTS_FILE) as file:
    experiments = json.load(file)
OUTPUT_FILE = EXPERIMENTS_FILE.replace("all_results.json", "all_results_2.json")

In [3]:
#EXPERIMENTS_FILE = "experiments/normal/005_init_lat_ind_best.json"
#with open(EXPERIMENTS_FILE) as file:
#    experiments = json.load(file)
#NOISE_RATIO = 0.50
#for experiment in experiments:
#    del experiment['auc_roc']
#    del experiment['auc_pr']
#    experiment['noise_type'] = "irrelevant_features"
#    experiment['noise_ratio'] =  NOISE_RATIO
#with open( f"experiments/noisy/003_irrelevant_features_{int(100*NOISE_RATIO)}.json", "w") as file:
#    json.dump(experiments, file)

In [4]:
# anomaly_type = "dependency"
# for experiment in experiments:
#    experiment["anomaly_type"] = anomaly_type
# with open(f"experiments/noisy/002_anomaly_study_{anomaly_type}.json", "w") as file:
#    json.dump(experiments,file)

In [5]:
#experiments = []
#path = "experiments/noisy/"
#for file in os.listdir(path):
#    if file.find("_irrelevant_features_") > -1:
#        print(path+file)
#        with open(path+file, "r") as file:
#            experiments += json.load(file)

In [6]:
# for experiment in experiments:
#    del experiment['auc_pr']
#    del experiment['auc_roc']

In [7]:
#with open(f"experiments/noisy/003_irrelevant_features_all_experiments.json", "w") as file:
#    json.dump(experiments,file)

In [8]:
#with open(f"experiments/noisy/003_irrelevant_features_all_experiments.json") as irr_file:
#    experiment_irr = json.load(irr_file)
#with open(f"experiments/noisy/002_anomaly_study_all_experiments.json") as ano_file:
#    experiment_ano = json.load(ano_file)

In [9]:
#experiments = experiment_irr +  experiment_ano

In [10]:
#with open(f"experiments/noisy/004_irrelevant_anomaly_all_experiments.json", "w") as file:
#    json.dump(experiments,file)

In [12]:
def create_anomaly_dataframe(dataset, realistic_synthetic_mode=None, noise_type=None, noise_ratio=None):
    datagenerator.dataset = dataset
    data = datagenerator.generator(
        la=1.00,
        realistic_synthetic_mode=realistic_synthetic_mode,
        stdscale=True,
        minmax=False,
        noise_type=noise_type,
        noise_ratio=noise_ratio
    )
    Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
    Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
    lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
    lb_test = torch.tensor(data["y_test"], dtype=torch.float32)

    return Y_train, Y_test, lb_train, lb_test

In [ ]:
for experiment in tqdm_notebook(experiments):
    print(f"Start Experimento: {experiment}")
    if "auc_roc" not in experiment.keys() and "anomaly_type" in experiment.keys():
        Y_train, Y_test, lb_train, lb_test = create_anomaly_dataframe(
            experiment["dataset"], 
            realistic_synthetic_mode=experiment["anomaly_type"]
        )
    if "auc_roc" not in experiment.keys() and "noise_type" in experiment.keys():
        Y_train, Y_test, lb_train, lb_test = create_anomaly_dataframe(
            experiment["dataset"],
            noise_type=experiment["noise_type"],
            noise_ratio=experiment["noise_ratio"],
        )

        n_train = len(Y_train)
        data_dim = Y_train.shape[1]
        latent_dim = experiment["latent_dim"]
        nn_layers = tuple(map(int, experiment["layers"].split(",")))
        kernel = experiment["kernel"]
        n_inducing = experiment["n_inducing"]

        n_epochs = 1000
        lr = 0.01
        batch_size = 128

        try:
            gplvm = AD_GPLVM(
                latent_dim, n_inducing, n_epochs, nn_layers, lr, batch_size
            )
            gplvm.fit(Y_train)
            score = gplvm.predict_score(Y_test)
            metrics = utils.metric(y_true=lb_test, y_score=score)

            experiment["auc_roc"] = metrics["aucroc"]
            experiment["auc_pr"] = metrics["aucpr"]
        except:
            experiment["auc_roc"] = 0.00
            experiment["auc_pr"] = 0.00

        experiment["lr"] = lr
        experiment["batch_size"] = batch_size
        experiment["n_epochs"] = n_epochs

        experiment["train_noise"] = float(lb_train.sum() / len(Y_train))

        with open(OUTPUT_FILE, "w") as file:
            json.dump(experiments, file)

/tmp/ipykernel_4768/2407570982.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for experiment in tqdm_notebook(experiments):


  0%|          | 0/423 [00:00<?, ?it/s]

Start Experimento: {'dataset': '01_ALOI', 'kernel': 'rbf', 'layers': '5,5', 'latent_dim': 13, 'lr': 0.01, 'batch_size': 128, 'n_inducing': 50, 'n_epochs': 1000, 'noise_type': 'irrelevant_features', 'noise_ratio': 0.01, 'auc_roc': 0.5001001061502952, 'auc_pr': 0.036067882485622316, 'train_noise': 0.030142856761813164}
Start Experimento: {'dataset': '02_annthyroid', 'kernel': 'rbf', 'layers': '10,10,10', 'latent_dim': 3, 'lr': 0.01, 'batch_size': 128, 'n_inducing': 50, 'n_epochs': 1000, 'noise_type': 'irrelevant_features', 'noise_ratio': 0.01, 'auc_roc': 0.6650937499999999, 'auc_pr': 0.1863903675599543, 'train_noise': 0.07420635223388672}
Start Experimento: {'dataset': '03_backdoor', 'kernel': 'rbf', 'layers': '15,15', 'latent_dim': 2, 'lr': 0.01, 'batch_size': 128, 'n_inducing': 50, 'n_epochs': 1000, 'noise_type': 'irrelevant_features', 'noise_ratio': 0.01, 'auc_roc': 0.7962579162238711, 'auc_pr': 0.29518064267691163, 'train_noise': 0.023571427911520004}
Start Experimento: {'dataset': '

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/copulas/multivariate/vine.py:73: UserWarning: Vines have not been fully tested on Python >= 3.8 and might produce wrong results.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/copulas/bivariate/base.py:162: RuntimeWarning: Data does not appear to be uniform.
  warnings.warn('Data does not appear to be uniform.', category=RuntimeWarning)


In [13]:
pd.DataFrame(experiments).to_csv(OUTPUT_FILE.replace(".json", ".csv"))